## Теоретическая часть


1. Ответьте на вопросы:  
В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной фильтрации?
Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы  




Колаборативная фильтрация основывается на оценках пользователя или на оценках других пользователей, а гибридная система включается в себя не только эти знания, но и например знания о пользователях или товарах.
Гибридную систему можно применять там где есть достаточно данных и ресурсов для ее построения, а так же имеется в этом целесообразность.


2.  Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/
Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить
Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

На мой взгляд на семинаре мы не разбирали как одновременно помещать в топ рекомендации из разных подгрупп («Премиумы», «Стандарт+» и «Стандарт», и бесплатные).

3. На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изучить оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:
1) Какой датасет используют авторы?
    1. MovieLens - содержит 10 миллионов записей о рейтинге 10681 фильме составленные 71567 пользователями. Все фильмы имеют описание по жанру и списку тегов. Для пары фильм-тег есть оценка соответствия от 0 до 1. Ожидается что с данным набором будет хорошо справляться модель матричной факторизации.
    2. CrossValidated - содержит информацию о 44200 вопросах и 188865 ответах полученных от 5953 пользователя. Каждому вопросу соответствует один или несколько из 1032 уникальных тегов. Так же набор содержит информацию о пользователях. Ожидается что с набором хорошо будет справляться Content-based модель.
2) Что используют в качестве признаков?
    1. В качестве признаков используют: теги, описательные признаки, признаки пользователей и признаки элементов.
3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко
    1. Матричная факторизация - традиционная модель матричной факторизации
    2. LSI-LR - гибридная модель основанная на контенте. Использует модели Latent Semantic Indexing(LSI) и линейную регрессию. Сначала для каждого пользователя с помощью LSI строиться вектор рекомендаций, после чего для каждого пользователя обучается своя модель логистической регрессии.
    3. LSI-UP - гибридная модель основанная на профилях пользователей, представленных в виде линейной комбинации векторов контента, к которым применяется LSI для получения итоговой неявных представлений пользователей и товаров. В начале строится матрица характеристик пользователей, где каждому пользователю соответствует вектор положительного взаимодействия с товаром. После чего к нормализованной матрице применяется усеченное сингулярное разложение, чтобы получить векторы неявных пользователей и неявных признаков.

## Практическая часть


### 2. Работа с признаками
У нас есть внешние данные. Что с ними не так? Чего не хватает?

Проведите исследование внешних данных и составьте какие-нибудь содержательные выводы.
Формально Вам нужно построить 3+ графиков (scatter plot, hist или что-то иное) и описать, что мы видим (например, товары такой-то категории более часто покупаются в следующие дни недели или пользователи с большим достатком предпочитают такие-то товары).
Исследуйте те закономерности, которые Вам интересно, чем менее тривиальный вывод получается, тем лучше!

### 3.LightFM
У этого алгоритма есть множество параметров (item/user_alpha, loss, no_components). Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способом и постройте графики) На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 2%
У Вас, скорее всего, возникнет проблема со временем обучения. Почему они возникают?
Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features[‘commodity_desc’].unique() - 300 уникальных категорий - это очень много)

### *Отбор признаков* *
Все данные категориальные, при ohe кодировании для товаров признаков становится невероятно много.
Какие стратегии отбора признаков в классическом ML Вы знаете? Применимы ли они тут?

Попробуйте какие-нибудь стратегии. Удалось ли улучшить качество?

 \* *задание необязательно*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightfm
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
#from implicit.als import AlternatingLeastSquares
#from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k as pr, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import prefilter_items, id_converter
from src.recommenders import get_similar_users_recommendation, get_similar_item

In [ ]:
data = pd.read_csv('./input/retail_train.csv')
item_features = pd.read_csv('./input/product.csv')
user_features = pd.read_csv('./input/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

current_week = data['week_no'].max()
test_week =  current_week - test_size_weeks

data_train = data[data['week_no'] < current_week]
data_test = data[data['week_no'] >= current_week]

data_train.head(2)

In [ ]:
item_features.head(2)

In [ ]:
user_features.head(2)

In [ ]:
data_train.head(2)

In [ ]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features, current_week, n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

In [ ]:
data_test = prefilter_items(data_test, item_features, current_week, n_popular=5000)

In [ ]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# filter non top-5000
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

In [ ]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float)
sparse_user_item_test = csr_matrix(test_user_item_matrix).tocsr()

In [ ]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

In [ ]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [ ]:
user_feat_lightfm.head(2)

In [ ]:
item_feat_lightfm.head()

In [ ]:
csr_user = csr_matrix(user_feat_lightfm.values).tocsr()
csr_item = csr_matrix(item_feat_lightfm.values).tocsr()
c_ui = coo_matrix(user_item_matrix)

In [ ]:
# Params:
# no_components – the dimensionality of the feature latent embeddings.
# k – for k-OS training, the k-th positive example will be selected from the n positive examples sampled for every user.
# n – for k-OS training, maximum number of positives sampled for each update.
# learning_schedule – one of ('adagrad', 'adadelta').
# loss – one of ('logistic', 'bpr', 'warp', 'warp-kos'): the loss function.
# learning_rate – initial learning rate for the adagrad learning schedule.
# rho – moving average coefficient for the adadelta learning schedule.
# epsilon – conditioning parameter for the adadelta learning schedule.
# item_alpha – L2 penalty on item features. Tip: setting this number too high can slow down training. One good way to check is if the final weights in the embeddings turned out to be mostly zero. The same idea applies to the user_alpha parameter.
# user_alpha – L2 penalty on user features.
# max_sampled – maximum number of negative samples used during WARP fitting. It requires a lot of sampling to find negative triplets for users that are already well represented by the model; this can lead to very long training times and overfitting. Setting this to a higher number will generally lead to longer training times, but may in some cases improve accuracy.
# random_state – The seed of the pseudo random number generator to use when shuffling the data and initializing the parameters.

# model = LightFM(no_components=10,
#                 loss='warp', # 'bpr'
#                 learning_rate=0.05,
#                 item_alpha=0.1, # смещение по товару
#                 user_alpha=0.1,
#                 random_state=42)


model = LightFM(no_components = 10,
                k = 5,
                n = 10,
                # learning_schedule = "adagrad",
                loss = "bpr",
                learning_rate = 0.05,
                # rho = 0.95,
                # epsilon = 1e-6,
                item_alpha = 0.5,
                user_alpha = 0.4,
                # max_sampled = 20,
                random_state = 42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
           sample_weight=c_ui,
           user_features=csr_user,
           item_features=csr_item,
           epochs=15)

test_precision = pr(model, sparse_user_item_test,
                    user_features=csr_user,
                    item_features=csr_item,
                    k=5).mean()
test_precision

In [ ]:
model = LightFM(no_components = 10,
                k = 5,
                n = 10,
                # learning_schedule = "adagrad",
                loss = "bpr",
                learning_rate = 0.05,
                # rho = 0.95,
                # epsilon = 1e-6,
                item_alpha = 0.5,
                user_alpha = 0.4,
                max_sampled = 12,
                random_state = 42)

0.011407543

In [ ]:
user_biases_emb = model.get_user_representations(
    features=csr_matrix(user_feat_lightfm.values).tocsr()
)

In [ ]:
item_biases_emb = model.get_item_representations(
    features=csr_matrix(item_feat_lightfm.values).tocsr()
)

In [ ]:
train_precision = pr(model, sparse_user_item,
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

In [ ]:
train_precision

In [ ]:
test_precision = pr(model, sparse_user_item_test,
                    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                    k=5).mean()
test_precision

In [ ]:
test_item_ids = np.arange(len(itemids))

predictions = model.predict(user_ids=6, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [5]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features, current_week, n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 88546 to 4937


In [7]:
data_test = prefilter_items(data_test, item_features, current_week, n_popular=5000)

In [8]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# filter non top-5000
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,-1,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15972298,15972565,15972790,16100266,16729299,16729415,16770156,16809649,17104444,17105058
user_id,,,,,,,,,,,,,,,,,,,,,
1,142.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float)
sparse_user_item_test = csr_matrix(test_user_item_matrix).tocsr()

In [11]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [13]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
item_feat_lightfm.head()

,manufacturer_2.0,manufacturer_5.0,manufacturer_16.0,manufacturer_20.0,manufacturer_26.0,manufacturer_33.0,manufacturer_35.0,manufacturer_36.0,manufacturer_42.0,manufacturer_43.0,...,curr_size_of_product_L 16 OZ,curr_size_of_product_L 7.75 OZ,curr_size_of_product_L 13.25 OZ,curr_size_of_product_LB,curr_size_of_product_N 12 OZ,curr_size_of_product_N 40 OZ,curr_size_of_product_PINT,curr_size_of_product_PK,curr_size_of_product_PT,curr_size_of_product_QT
item_id,,,,,,,,,,,,,,,,,,,,,
-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
818981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
819255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
819308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
819400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
csr_user = csr_matrix(user_feat_lightfm.values).tocsr()
csr_item = csr_matrix(item_feat_lightfm.values).tocsr()
c_ui = coo_matrix(user_item_matrix)

In [16]:
# Params:
# no_components – the dimensionality of the feature latent embeddings.
# k – for k-OS training, the k-th positive example will be selected from the n positive examples sampled for every user.
# n – for k-OS training, maximum number of positives sampled for each update.
# learning_schedule – one of ('adagrad', 'adadelta').
# loss – one of ('logistic', 'bpr', 'warp', 'warp-kos'): the loss function.
# learning_rate – initial learning rate for the adagrad learning schedule.
# rho – moving average coefficient for the adadelta learning schedule.
# epsilon – conditioning parameter for the adadelta learning schedule.
# item_alpha – L2 penalty on item features. Tip: setting this number too high can slow down training. One good way to check is if the final weights in the embeddings turned out to be mostly zero. The same idea applies to the user_alpha parameter.
# user_alpha – L2 penalty on user features.
# max_sampled – maximum number of negative samples used during WARP fitting. It requires a lot of sampling to find negative triplets for users that are already well represented by the model; this can lead to very long training times and overfitting. Setting this to a higher number will generally lead to longer training times, but may in some cases improve accuracy.
# random_state – The seed of the pseudo random number generator to use when shuffling the data and initializing the parameters.

# model = LightFM(no_components=10,
#                 loss='warp', # 'bpr'
#                 learning_rate=0.05,
#                 item_alpha=0.1, # смещение по товару
#                 user_alpha=0.1,
#                 random_state=42)


model = LightFM(no_components = 10,
                k = 5,
                n = 10,
                # learning_schedule = "adagrad",
                loss = "bpr",
                learning_rate = 0.05,
                # rho = 0.95,
                # epsilon = 1e-6,
                item_alpha = 0.5,
                user_alpha = 0.4,
                # max_sampled = 20,
                random_state = 42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
           sample_weight=c_ui,
           user_features=csr_user,
           item_features=csr_item,
           epochs=15)

test_precision = pr(model, sparse_user_item_test,
                    user_features=csr_user,
                    item_features=csr_item,
                    k=5).mean()
test_precision

0.011407543

In [17]:
model = LightFM(no_components = 10,
                k = 5,
                n = 10,
                # learning_schedule = "adagrad",
                loss = "bpr",
                learning_rate = 0.05,
                # rho = 0.95,
                # epsilon = 1e-6,
                item_alpha = 0.5,
                user_alpha = 0.4,
                max_sampled = 12,
                random_state = 42)

0.011407543

0.011407543

In [18]:
user_biases_emb = model.get_user_representations(
    features=csr_matrix(user_feat_lightfm.values).tocsr()
)

ValueError: You must fit the model before trying to obtain predictions.

In [ ]:
item_biases_emb = model.get_item_representations(
    features=csr_matrix(item_feat_lightfm.values).tocsr()
)

In [ ]:
train_precision = pr(model, sparse_user_item,
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

In [ ]:
train_precision

In [ ]:
test_precision = pr(model, sparse_user_item_test,
                    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                    k=5).mean()
test_precision

In [ ]:
test_item_ids = np.arange(len(itemids))

predictions = model.predict(user_ids=6, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)